<a href="https://colab.research.google.com/github/harryguiacorn/colab/blob/main/Ichimoku_chart_display.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step by step

In [ ]:
!pip install yfinance pandas plotly

## Download raw data and load into a data frame



In [ ]:
import yfinance as yf
import pandas as pd
import datetime

# Define the ticker symbol (SPY) and end date
ticker_symbol = "SPY"
look_back_month = 8
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=look_back_month*30)  # x months of data

# Download historical data using yfinance
data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Format the dates as strings
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

# Save the data to a CSV file with dates in the filename
csv_filename = f"{ticker_symbol}_{start_date_str}-{end_date_str}_daily_data"
csv_filename_full = f"{csv_filename}.csv"

print(f"Data for {ticker_symbol} saved to {csv_filename_full}")
data.to_csv(csv_filename_full)

[*********************100%%**********************]  1 of 1 completed
Data for SPY saved to SPY_2023-03-06-2023-11-01_daily_data.csv


In [ ]:

# Load the CSV data
data = pd.read_csv(csv_filename_full)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)


## Use Tapy to generate Ichimoku data

In [ ]:
!pip install tapy
import pandas as pd
from tapy import Indicators
from datetime import datetime

# Apply Ichimoku analysis
# ichimoku = ta.Ichimoku()
# data = ichimoku(data)

__df_ichimoku = Indicators(data)
__df_ichimoku.ichimoku_kinko_hyo()  # column_name_kijun_sen="K Line"

# Save the data with Ichimoku analysis to a new CSV file
output_csv_filename = f"{csv_filename}_ichimoku_tapy"
output_csv_filename_full = f"{output_csv_filename}.csv"
data_ichimoku_tapy = __df_ichimoku.df
data_ichimoku_tapy.to_csv(output_csv_filename_full)
# print(data_ichimoku_tapy.head)
print(f"Ichimoku analysis for SPY data saved to {output_csv_filename_full}")


Ichimoku analysis for SPY data saved to SPY_2023-03-06-2023-11-01_daily_data_ichimoku_tapy.csv


## Use Finta to generate Ichimoku data

In [ ]:
!pip install finta

from finta import TA

# Finta works but doesn't produce future senkou a or b.
# df = pd.read_csv(csv_filename_full)
output_csv_filename = f"{csv_filename}_ichimoku_finta"
output_csv_filename_full = f"{output_csv_filename}.csv"
data_ichimoku_finta = TA.ICHIMOKU(data).to_csv(output_csv_filename_full)

## Plot on a chart

## Line Chart

In [ ]:
import plotly.express as px

# Create an interactive chart
fig = px.line(data_ichimoku_tapy, x=data.index, y='Close', title='SPY Daily Closing Prices')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Close Price')
fig.show()

## Interactive Chart

In [ ]:
!pip install -U kaleido

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio


# data["tenkan_sen"] = data_ichimoku_tapy["tenkan_sen"]
# data["kijun_sen"] = data_ichimoku_tapy["kijun_sen"]
# data["senkou_span_a"] = data_ichimoku_tapy["senkou_span_a"]
# data["senkou_span_b"] = data_ichimoku_tapy["senkou_span_b"]

# Create the interactive Ichimoku cloud chart
fig = go.Figure()

# Add candlestick chart
fig.add_trace(
    go.Candlestick(
        x=data_ichimoku_tapy.index,
        open=data_ichimoku_tapy["Open"],
        high=data_ichimoku_tapy["High"],
        low=data_ichimoku_tapy["Low"],
        close=data_ichimoku_tapy["Close"],
        name="Candlesticks",
    )
)

# Add the Tenkan-sen and Kijun-sen lines
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["tenkan_sen"],
        mode="lines",
        name="Tenkan-sen",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["kijun_sen"],
        mode="lines",
        name="Kijun-sen",
        line=dict(color="red"),
    )
)

# Add Senkou Span A and Senkou Span B as filled areas
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["senkou_span_a"],
        fillcolor="rgba(0,255,0,0.2)",
        line=dict(width=2),
        name="Senkou Span A",
    )
)
fig.add_trace(
    go.Scatter(
        x=data_ichimoku_tapy.index,
        y=data_ichimoku_tapy["senkou_span_b"],
        fill="tonexty",
        fillcolor="rgba(255,0,0,0.2)",
        line=dict(width=2),
        name="Senkou Span B",
    )
)  # fill area between trace0 and trace1

# Update layout
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
    title=f"{ticker_symbol} Ichimoku Cloud Chart",
    xaxis_rangeslider_visible=False
)

# Filter out empty data points on the x-axis
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
fig.update_yaxes(fixedrange=False)
# fig.update_yaxes(autorange=True)  # Set the y-axis to auto-range
# fig.update_layout(yaxis_range=[-3,3])

# Show the interactive chart
fig.show()

# Save the chart as an HTML file
fig.write_html(f"{ticker_symbol}_{start_date_str}-{end_date_str}_ichimoku_chart.html")

# Save the chart as a JPEG image
file_name = f"{ticker_symbol}_{start_date_str}-{end_date_str}_ichimoku_chart.jpg"
fig.write_image(file_name, width = 1500, scale = 1)




# Encapulate into a class

In [1]:
!pip install -U yfinance pandas plotly tapy kaleido

## Functions

In [2]:
# Create asset list folder
from os.path import isdir
import os

def createDataFolder(folder_name="data"):
    # Create data folder
    print(f"{folder_name}")
    try:
        if not isdir(folder_name):
            os.mkdir(folder_name)
        else:
            print("Data folder already exists")
    except FileExistsError as errFile:
        print("Error:", errFile)

def createNestedFolder(__rootFolder = "", __nestedFolder = "", __nestedFolder2 = ""):
  # Specify the path to the root directory
  root_dir = __rootFolder

  # Create the root directory
  os.makedirs(root_dir, exist_ok=True)

  # Create nested directories within the root directory
  nested_dir1 = os.path.join(root_dir, __nestedFolder)

  os.makedirs(nested_dir1, exist_ok=True)

  # You can create further nested directories in the same way
  nested_dir2 = os.path.join(nested_dir1, __nestedFolder2)
  os.makedirs(nested_dir2, exist_ok=True)

if __name__ == "__main__":
    createDataFolder("asset_list")


asset_list
Data folder already exists


## Classes

In [3]:
import yfinance as yf
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.io as pio
from tapy import Indicators

class IchimokuAnalyzer:
    def __init__(self, ticker_symbol, look_back_month=8):
        self.ticker_symbol = ticker_symbol
        self.look_back_month = look_back_month
        self.end_date = datetime.datetime.now() + datetime.timedelta(days=1)  # Adjusted to include today
        self.start_date = self.end_date - datetime.timedelta(days=look_back_month * 30)
        self.csv_filename = f"{self.ticker_symbol}_{self.start_date.strftime('%Y-%m-%d')}_{self.end_date.strftime('%Y-%m-%d')}_daily_data.csv"

    def download_data(self, __rootFolder = "", __nestedFolder = "", __nestedFolder2 = ""):
        # print(f"{self.ticker_symbol}")
        createNestedFolder(__rootFolder, __nestedFolder, __nestedFolder2)
        data = yf.download(self.ticker_symbol, start=self.start_date, end=self.end_date)
        data.to_csv(f"{__rootFolder}/{__nestedFolder}/{self.csv_filename}")
        return data

    def apply_ichimoku_analysis(self, __rootFolder = "", __nestedFolder = ""):
        data = pd.read_csv(f"{__rootFolder}/{__nestedFolder}/{self.csv_filename}")
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)

        df_ichimoku = Indicators(data)
        df_ichimoku.ichimoku_kinko_hyo()
        self.data_ichimoku_tapy = df_ichimoku.df

    def create_ichimoku_chart(self):
        fig = go.Figure()
        fig.add_trace(
            go.Candlestick(
                x=self.data_ichimoku_tapy.index,
                open=self.data_ichimoku_tapy["Open"],
                high=self.data_ichimoku_tapy["High"],
                low=self.data_ichimoku_tapy["Low"],
                close=self.data_ichimoku_tapy["Close"],
                name="Candlesticks",
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["tenkan_sen"],
                mode="lines",
                name="Tenkan-sen",
                line=dict(color="blue"),
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["kijun_sen"],
                mode="lines",
                name="Kijun-sen",
                line=dict(color="red"),
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["senkou_span_a"],
                fillcolor="rgba(0,255,0,0.2)",
                line=dict(width=2),
                name="Senkou Span A",
            )
        )
        fig.add_trace(
            go.Scatter(
                x=self.data_ichimoku_tapy.index,
                y=self.data_ichimoku_tapy["senkou_span_b"],
                fill="tonexty",
                fillcolor="rgba(255,0,0,0.2)",
                line=dict(width=2),
                name="Senkou Span B",
            )
        )
        fig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list(
                        [
                            dict(count=1, label="1m", step="month", stepmode="backward"),
                            dict(count=6, label="6m", step="month", stepmode="backward"),
                            dict(count=1, label="YTD", step="year", stepmode="todate"),
                            dict(count=1, label="1y", step="year", stepmode="backward"),
                            dict(step="all"),
                        ]
                    )
                ),
                rangeslider=dict(visible=True),
                type="date",
            ),
            title=f"{self.ticker_symbol} Ichimoku Cloud Chart",
            xaxis_rangeslider_visible=False
        )
        fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
        fig.update_yaxes(fixedrange=False)
        self.fig = fig

    def show_chart(self):
        self.fig.show()

    def save_chart_as_html(self, __rootFolder = "", __nestedFolder = "", __nestedFolder2 = ""):
        createNestedFolder(__rootFolder, __nestedFolder, __nestedFolder2)
        self.fig.write_html(f"{__rootFolder}/{__nestedFolder}/{self.ticker_symbol}_{self.start_date.strftime('%Y-%m-%d')}_{self.end_date.strftime('%Y-%m-%d')}_ichimoku_chart.html")

    def save_chart_as_image(self, __rootFolder = "", __nestedFolder = "", __nestedFolder2 = "", width=1500, scale=1):
        createNestedFolder(__rootFolder, __nestedFolder, __nestedFolder2)
        self.fig.write_image(f"{__rootFolder}/{__nestedFolder}/{self.ticker_symbol}_{self.start_date.strftime('%Y-%m-%d')}_{self.end_date.strftime('%Y-%m-%d')}_ichimoku_chart.jpg", width=width, scale=scale)

# Usage example:
# if __name__ == "__main__":
#   analyzer = IchimokuAnalyzer("SPY", look_back_month=8)
#   analyzer.download_data("data", "DowJones30")
#   analyzer.apply_ichimoku_analysis("data", "DowJones30")
#   analyzer.create_ichimoku_chart()
#   analyzer.show_chart()
#   analyzer.save_chart_as_html("html", "DowJones30")
#   analyzer.save_chart_as_image("jpg", "DowJones30")


## Get Dow Jones 30 stock tickers

In [4]:
# GetSymbolDowJones30.py
import pandas as pd

class Model(object):
    def __init__(
        self,
        __url,
        __fileNameCSV,
        __readHtmlMatch="",
    ) -> None:
        self.url = __url
        self.fileNameCSV = __fileNameCSV
        self.readHtmlMatch = __readHtmlMatch
        self.df_list = None
        self.df = None

    @property
    def df_list(self):
        return self.__df_list

    @df_list.setter
    def df_list(self, __df_list):
        self.__df_list = __df_list

    def readHtml(self):
        self.df_list = pd.read_html(
            self.url,
            match=self.readHtmlMatch,
        )[0]

        # return type is list[DataFrame]
        print(f"Total tables: {len(self.df_list)}")
        return self.df_list

    def cleanData(self):
        __df_list = self.df_list
        self.df = __df_list
        self.df.rename(
            columns={
                "Company": "name",
                "Symbol": "symbol",
            },
            inplace=True,
        )
        self.df["symbol"] = self.df["symbol"].str.replace(
            ".", "-", regex=False
        )

    def saveData(self):
        __columns = ["symbol", "name"]
        # print(type(self.df))
        # print(self.df)
        # print(self.df[__columns])
        self.df.to_csv(
            self.fileNameCSV,
            columns=__columns,
            index=False,
        )
        return self.df[__columns]


class View(object):

    pass


class Control(object):
    def __init__(self, model, view) -> None:
        self.model = model
        self.view = view

    def main(self):
        self.readHtml()
        # print(self.model.df_list)
        self.cleanData()
        self.saveData()

    def readHtml(self, __match=""):
        self.model.readHtml()

    def cleanData(self):
        self.model.cleanData()

    def saveData(self):
        self.model.saveData()


def main(__fetch_symbols_latest=True):
    if __fetch_symbols_latest is False:
        return
    _model = Model(
        "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components",
        "asset_list/DowJones30.csv",
        "DJIA component companies",
    )

    _control = Control(_model, View())
    _control.main()


if __name__ == "__main__":
    main()



Total tables: 30


In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
file_path = "asset_list/DowJones30.csv"
df = pd.read_csv(file_path)

def run_ichimokuAnalyser(symbol):
    # print(f"run_ichimokuAnalyser_{symbol}")
    analyzer = IchimokuAnalyzer(symbol, look_back_month=8)
    analyzer.download_data("data", "DowJones30")
    analyzer.apply_ichimoku_analysis("data", "DowJones30")
    analyzer.create_ichimoku_chart()
    analyzer.show_chart()
    analyzer.save_chart_as_html("html", "DowJones30")
    analyzer.save_chart_as_image("jpg", "DowJones30")
# Access and iterate through the first column (assuming it's named "Column1") using enumerate
# for symbol, name in enumerate(df.items()):

df['symbol'].apply(run_ichimokuAnalyser)



# Put all charts into a pdf

In [51]:
!pip install -U img2pdf


In [58]:
import os
import img2pdf

class ImageToPdfConverter:
  def __init__(self, directory_path='.', outputPath='output.pdf'):
      self.directory_path = directory_path
      self.outputPath = outputPath

  def convert(self, pdf_folder ="", pdf_nested_folder =""):
    # List all files in the directory and filter only JPEG images (ending with ".jpg")
    image_files = [f"{self.directory_path}/{i}" for i in os.listdir(self.directory_path) if i.endswith(".jpg")]

    # Convert the list of JPEG images to a single PDF file
    pdf_data = img2pdf.convert(image_files)

    createNestedFolder(pdf_folder, pdf_nested_folder)
    # Write the PDF content to a file (make sure you have write permissions for the specified file)
    with open(f"{pdf_folder}/{pdf_nested_folder}/{self.outputPath}", "wb") as file:
      file.write(pdf_data)

if __name__ == "__main__":
  __imgToPdf = ImageToPdfConverter('jpg/DowJones30/', 'DowJones30StocksIchimoku.pdf')
  __imgToPdf.convert("pdf", "DowJones30")
